In [33]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

In [38]:

file_path = r"C:\Users\KIIT\Downloads\kaggle\input\WA_Fn-UseC_-Telco-Customer-Churn.csv"
df = pd.read_csv(file_path)


In [40]:
import os
os.listdir()

['.ipynb_checkpoints',
 'ex.ipynb',
 'lab2.ipynb',
 'Lab6.ipynb',
 'Lab6b.ipynb',
 'Lab6c.ipynb',
 'Lab7.ipynb',
 'Lab8.ipynb',
 'logistic .ipynb',
 'Untitled.ipynb',
 'Untitled1.ipynb']

In [42]:
df.head

<bound method NDFrame.head of       customerID  gender  SeniorCitizen Partner Dependents  tenure  \
0     7590-VHVEG  Female              0     Yes         No       1   
1     5575-GNVDE    Male              0      No         No      34   
2     3668-QPYBK    Male              0      No         No       2   
3     7795-CFOCW    Male              0      No         No      45   
4     9237-HQITU  Female              0      No         No       2   
...          ...     ...            ...     ...        ...     ...   
7038  6840-RESVB    Male              0     Yes        Yes      24   
7039  2234-XADUH  Female              0     Yes        Yes      72   
7040  4801-JZAZL  Female              0     Yes        Yes      11   
7041  8361-LTMKD    Male              1     Yes         No       4   
7042  3186-AJIEK    Male              0      No         No      66   

     PhoneService     MultipleLines InternetService OnlineSecurity  ...  \
0              No  No phone service             DSL   

In [44]:
df.tail

<bound method NDFrame.tail of       customerID  gender  SeniorCitizen Partner Dependents  tenure  \
0     7590-VHVEG  Female              0     Yes         No       1   
1     5575-GNVDE    Male              0      No         No      34   
2     3668-QPYBK    Male              0      No         No       2   
3     7795-CFOCW    Male              0      No         No      45   
4     9237-HQITU  Female              0      No         No       2   
...          ...     ...            ...     ...        ...     ...   
7038  6840-RESVB    Male              0     Yes        Yes      24   
7039  2234-XADUH  Female              0     Yes        Yes      72   
7040  4801-JZAZL  Female              0     Yes        Yes      11   
7041  8361-LTMKD    Male              1     Yes         No       4   
7042  3186-AJIEK    Male              0      No         No      66   

     PhoneService     MultipleLines InternetService OnlineSecurity  ...  \
0              No  No phone service             DSL   

In [46]:
missing_values = df.isnull().sum()
print(missing_values)

customerID          0
gender              0
SeniorCitizen       0
Partner             0
Dependents          0
tenure              0
PhoneService        0
MultipleLines       0
InternetService     0
OnlineSecurity      0
OnlineBackup        0
DeviceProtection    0
TechSupport         0
StreamingTV         0
StreamingMovies     0
Contract            0
PaperlessBilling    0
PaymentMethod       0
MonthlyCharges      0
TotalCharges        0
Churn               0
dtype: int64


In [48]:
print('no missing values')

no missing values


In [50]:
# Label Encoding for 'gender', 'Partner', 'Dependents', etc. (binary categories)
df['gender'] = df['gender'].map({'Male': 1, 'Female': 0})
df['Partner'] = df['Partner'].map({'Yes': 1, 'No': 0})
df['Dependents'] = df['Dependents'].map({'Yes': 1, 'No': 0})
df['PhoneService'] = df['PhoneService'].map({'Yes': 1, 'No': 0})
df['MultipleLines'] = df['MultipleLines'].map({'Yes': 1, 'No': 0, 'No phone service': 0})  # If there's a 'No phone service' option
df['OnlineSecurity'] = df['OnlineSecurity'].map({'Yes': 1, 'No': 0, 'No internet service': 0})
df['OnlineBackup'] = df['OnlineBackup'].map({'Yes': 1, 'No': 0, 'No internet service': 0})
df['DeviceProtection'] = df['DeviceProtection'].map({'Yes': 1, 'No': 0, 'No internet service': 0})
df['TechSupport'] = df['TechSupport'].map({'Yes': 1, 'No': 0, 'No internet service': 0})
df['StreamingTV'] = df['StreamingTV'].map({'Yes': 1, 'No': 0, 'No internet service': 0})
df['StreamingMovies'] = df['StreamingMovies'].map({'Yes': 1, 'No': 0, 'No internet service': 0})
df['PaperlessBilling'] = df['PaperlessBilling'].map({'Yes': 1, 'No': 0})
df['Churn'] = df['Churn'].map({'Yes': 1, 'No': 0})

# One-Hot Encoding for 'Contract' and 'PaymentMethod' (non-binary categories)
df = pd.get_dummies(df, columns=['Contract', 'PaymentMethod'], drop_first=True)

In [52]:
# Convert 'TotalCharges' to numeric, forcing errors to NaN
df['TotalCharges'] = pd.to_numeric(df['TotalCharges'], errors='coerce')

# Check for missing values after conversion
missing_values = df.isnull().sum()

# Impute missing values in 'TotalCharges' using the median
df.loc[:, 'TotalCharges'] = df['TotalCharges'].fillna(df['TotalCharges'].median())

# Verify missing values are handled
df.isnull().sum()

customerID                               0
gender                                   0
SeniorCitizen                            0
Partner                                  0
Dependents                               0
tenure                                   0
PhoneService                             0
MultipleLines                            0
InternetService                          0
OnlineSecurity                           0
OnlineBackup                             0
DeviceProtection                         0
TechSupport                              0
StreamingTV                              0
StreamingMovies                          0
PaperlessBilling                         0
MonthlyCharges                           0
TotalCharges                             0
Churn                                    0
Contract_One year                        0
Contract_Two year                        0
PaymentMethod_Credit card (automatic)    0
PaymentMethod_Electronic check           0
PaymentMeth

In [54]:
from sklearn.preprocessing import LabelEncoder

# Identify categorical columns (excluding 'customerID' which is not useful for modeling)
categorical_columns = df.select_dtypes(include=['object']).columns.tolist()
categorical_columns.remove('customerID')  # Exclude ID column

# Encode binary categorical columns using Label Encoding
binary_columns = ['gender', 'Partner', 'Dependents', 'PaperlessBilling', 'Churn']
label_encoder = LabelEncoder()
for col in binary_columns:
    df[col] = label_encoder.fit_transform(df[col])

# Apply one-hot encoding for nominal categorical variables
df = pd.get_dummies(df, columns=[col for col in categorical_columns if col not in binary_columns], drop_first=True)

# Display updated dataset structure
df.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,OnlineSecurity,OnlineBackup,...,MonthlyCharges,TotalCharges,Churn,Contract_One year,Contract_Two year,PaymentMethod_Credit card (automatic),PaymentMethod_Electronic check,PaymentMethod_Mailed check,InternetService_Fiber optic,InternetService_No
0,7590-VHVEG,0,0,1,0,1,0,0,0,1,...,29.85,29.85,0,False,False,False,True,False,False,False
1,5575-GNVDE,1,0,0,0,34,1,0,1,0,...,56.95,1889.50,0,True,False,False,False,True,False,False
2,3668-QPYBK,1,0,0,0,2,1,0,1,1,...,53.85,108.15,1,False,False,False,False,True,False,False
3,7795-CFOCW,1,0,0,0,45,0,0,1,0,...,42.30,1840.75,0,True,False,False,False,False,False,False
4,9237-HQITU,0,0,0,0,2,1,0,0,0,...,70.70,151.65,1,False,False,False,True,False,True,False


In [56]:
from sklearn.preprocessing import MinMaxScaler
# Apply Min-Max Scaling to numerical columns
scaler = MinMaxScaler()
numerical_columns = ['tenure', 'MonthlyCharges', 'TotalCharges']
df[numerical_columns] = scaler.fit_transform(df[numerical_columns])

# Display dataset after scaling
df.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,OnlineSecurity,OnlineBackup,...,MonthlyCharges,TotalCharges,Churn,Contract_One year,Contract_Two year,PaymentMethod_Credit card (automatic),PaymentMethod_Electronic check,PaymentMethod_Mailed check,InternetService_Fiber optic,InternetService_No
0,7590-VHVEG,0,0,1,0,0.013889,0,0,0,1,...,0.115423,0.001275,0,False,False,False,True,False,False,False
1,5575-GNVDE,1,0,0,0,0.472222,1,0,1,0,...,0.385075,0.215867,0,True,False,False,False,True,False,False
2,3668-QPYBK,1,0,0,0,0.027778,1,0,1,1,...,0.354229,0.010310,1,False,False,False,False,True,False,False
3,7795-CFOCW,1,0,0,0,0.625000,0,0,1,0,...,0.239303,0.210241,0,True,False,False,False,False,False,False
4,9237-HQITU,0,0,0,0,0.027778,1,0,0,0,...,0.521891,0.015330,1,False,False,False,True,False,True,False


In [58]:
print(df.columns)  # Check available column names

Index(['customerID', 'gender', 'SeniorCitizen', 'Partner', 'Dependents',
       'tenure', 'PhoneService', 'MultipleLines', 'OnlineSecurity',
       'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV',
       'StreamingMovies', 'PaperlessBilling', 'MonthlyCharges', 'TotalCharges',
       'Churn', 'Contract_One year', 'Contract_Two year',
       'PaymentMethod_Credit card (automatic)',
       'PaymentMethod_Electronic check', 'PaymentMethod_Mailed check',
       'InternetService_Fiber optic', 'InternetService_No'],
      dtype='object')


In [60]:
from sklearn.model_selection import train_test_split

# Define target variable and features
X = df.drop(columns=['Churn'])  # Churn is the target
y = df['Churn']

# Split dataset into training (80%) and testing (20%) sets
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# Display dataset shapes
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((5634, 24), (1409, 24), (5634,), (1409,))